# Ingeniería de Características
## Predicción de Clientes Fidelizables en E-commerce

### Objetivos de la Ingeniería de Características

Este notebook se enfoca en la creación de características a nivel de cliente que permitan predecir la fidelización. Los objetivos específicos incluyen:

1. **Implementación de métricas RFM**
   - Recency: Días desde la última compra
   - Frequency: Número de transacciones
   - Monetary: Valor monetario total

2. **Creación de variables de comportamiento**
   - Patrones de compra temporales
   - Diversidad de productos
   - Consistencia en el gasto

3. **Definición de la variable objetivo**
   - Criterios de fidelización basados en literatura
   - Validación de la definición con datos
   - Análisis de balance de clases

### Marco Teórico RFM

El análisis RFM es una técnica establecida en marketing directo que segmenta clientes basándose en:
- **Recency**: Clientes que compraron recientemente tienen mayor probabilidad de comprar nuevamente
- **Frequency**: Clientes frecuentes muestran mayor lealtad
- **Monetary**: Clientes de alto valor son más rentables de retener

In [5]:
# Configuración inicial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("Librerías cargadas para ingeniería de características")
print(f"Fecha de procesamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Librerías cargadas para ingeniería de características
Fecha de procesamiento: 2025-09-01 13:40:25


In [6]:
# Cargar datos limpios del notebook anterior
try:
    df = pd.read_csv('../data/processed/online_retail_clean.csv')
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    print(f"Dataset limpio cargado: {df.shape[0]:,} registros")
except FileNotFoundError:
    print("Archivo de datos limpios no encontrado. Ejecutar notebook 02 primero.")
    # Crear dataset sintético para continuidad
    np.random.seed(42)
    n_samples = 7500  # Dataset limpio sería menor
    
    df = pd.DataFrame({
        'InvoiceNo': [f'53{i:04d}' for i in range(n_samples)],
        'StockCode': np.random.choice(['85123A', '71053', '84406B'], n_samples),
        'Description': np.random.choice(['WHITE HANGING HEART', 'WHITE METAL LANTERN'], n_samples),
        'Quantity': np.random.poisson(3, n_samples) + 1,
        'InvoiceDate': pd.date_range('2010-12-01', '2011-12-09', periods=n_samples),
        'UnitPrice': np.random.gamma(2, 2, n_samples),
        'CustomerID': np.random.choice(range(12346, 18287), n_samples),
        'Country': np.random.choice(['United Kingdom', 'France', 'Germany'], n_samples, p=[0.7, 0.15, 0.15]),
        'Revenue': None
    })
    
    df['Revenue'] = df['Quantity'] * df['UnitPrice']
    print(f"Dataset sintético creado: {df.shape[0]:,} registros")

print(f"Período de análisis: {df['InvoiceDate'].min()} a {df['InvoiceDate'].max()}")
print(f"Clientes únicos: {df['CustomerID'].nunique():,}")

Dataset limpio cargado: 397,884 registros
Período de análisis: 2010-12-01 08:26:00 a 2011-12-09 12:50:00
Clientes únicos: 4,338


## 1. Cálculo de Métricas RFM

### Metodología de Cálculo

Las métricas RFM se calculan a nivel de cliente utilizando toda la historia transaccional disponible:

- **Recency**: Días transcurridos desde la última compra hasta la fecha de referencia
- **Frequency**: Número total de transacciones únicas (facturas) por cliente
- **Monetary**: Suma total del valor de todas las compras del cliente

In [7]:
# Cálculo de métricas RFM
print("CÁLCULO DE MÉTRICAS RFM")
print("=" * 30)

# Definir fecha de referencia (día después de la última transacción)
reference_date = df['InvoiceDate'].max() + timedelta(days=1)
print(f"Fecha de referencia para Recency: {reference_date.strftime('%Y-%m-%d')}")

# Calcular métricas RFM por cliente
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (reference_date - x.max()).days,  # Recency
    'InvoiceNo': 'nunique',  # Frequency (transacciones únicas)
    'Revenue': 'sum'  # Monetary
}).reset_index()

# Renombrar columnas
rfm.columns = ['CustomerID', 'Recency', 'Frequency', 'Monetary']

print(f"Métricas RFM calculadas para {len(rfm):,} clientes")
print(f"\nEstadísticas descriptivas de RFM:")
display(rfm[['Recency', 'Frequency', 'Monetary']].describe())

# Verificar distribuciones
print(f"\nRangos de valores:")
print(f"Recency: {rfm['Recency'].min()} - {rfm['Recency'].max()} días")
print(f"Frequency: {rfm['Frequency'].min()} - {rfm['Frequency'].max()} transacciones")
print(f"Monetary: £{rfm['Monetary'].min():.2f} - £{rfm['Monetary'].max():.2f}")

CÁLCULO DE MÉTRICAS RFM
Fecha de referencia para Recency: 2011-12-10
Métricas RFM calculadas para 4,338 clientes

Estadísticas descriptivas de RFM:


,Recency,Frequency,Monetary
count,4338.000000,4338.000000,4338.000000
mean,92.536422,4.272015,2054.266460
std,100.014169,7.697998,8989.230441
min,1.000000,1.000000,3.750000
25%,18.000000,1.000000,307.415000
50%,51.000000,2.000000,674.485000
75%,142.000000,5.000000,1661.740000
max,374.000000,209.000000,280206.020000



Rangos de valores:
Recency: 1 - 374 días
Frequency: 1 - 209 transacciones
Monetary: £3.75 - £280206.02


In [8]:
# Visualización de distribuciones RFM
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Distribución de Recency', 'Distribución de Frequency', 
                   'Distribución de Monetary', 'Correlación RFM'),
    specs=[[{"type": "histogram"}, {"type": "histogram"}],
           [{"type": "histogram"}, {"type": "heatmap"}]]
)

# Recency distribution
fig.add_trace(
    go.Histogram(x=rfm['Recency'], name='Recency', nbinsx=30, 
                marker_color='lightblue', opacity=0.7),
    row=1, col=1
)

# Frequency distribution
fig.add_trace(
    go.Histogram(x=rfm['Frequency'], name='Frequency', nbinsx=30,
                marker_color='lightgreen', opacity=0.7),
    row=1, col=2
)

# Monetary distribution (log scale for better visualization)
fig.add_trace(
    go.Histogram(x=np.log1p(rfm['Monetary']), name='Log(Monetary)', nbinsx=30,
                marker_color='lightcoral', opacity=0.7),
    row=2, col=1
)

# Correlation heatmap
correlation_matrix = rfm[['Recency', 'Frequency', 'Monetary']].corr()
fig.add_trace(
    go.Heatmap(z=correlation_matrix.values,
              x=correlation_matrix.columns,
              y=correlation_matrix.columns,
              colorscale='RdBu',
              zmid=0,
              text=correlation_matrix.round(3).values,
              texttemplate="%{text}",
              textfont={"size":12}),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Análisis de Distribuciones RFM", showlegend=False)
fig.show()

# Análisis de correlaciones
print("\nAnálisis de correlaciones RFM:")
print(correlation_matrix.round(3))


Análisis de correlaciones RFM:
           Recency  Frequency  Monetary
Recency      1.000     -0.261    -0.122
Frequency   -0.261      1.000     0.554
Monetary    -0.122      0.554     1.000
